In [31]:
# for data handling and viz
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# for machine learning
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score

# other
import warnings; warnings.filterwarnings('ignore')

In [19]:
%cd '/content/drive/MyDrive/Colab Notebooks/Crypto closing price/'

/content/drive/MyDrive/Colab Notebooks/Crypto closing price


In [20]:
df_train = pd.read_csv('Data/Train.csv').fillna(0)
df_test = pd.read_csv('Data/Test.csv').fillna(0)

In [21]:
df_train.head()

,id,asset_id,open,high,low,volume,market_cap,url_shares,unique_url_shares,reddit_posts,reddit_posts_score,reddit_comments,reddit_comments_score,tweets,tweet_spam,tweet_followers,tweet_quotes,tweet_retweets,tweet_replies,tweet_favorites,tweet_sentiment1,tweet_sentiment2,tweet_sentiment3,tweet_sentiment4,tweet_sentiment5,tweet_sentiment_impact1,tweet_sentiment_impact2,tweet_sentiment_impact3,tweet_sentiment_impact4,tweet_sentiment_impact5,social_score,average_sentiment,news,price_score,social_impact_score,correlation_rank,galaxy_score,volatility,market_cap_rank,percent_change_24h_rank,volume_24h_rank,social_volume_24h_rank,social_score_24h_rank,medium,youtube,social_volume,percent_change_24h,market_cap_global,close
0,ID_322qz6,1,9422.849081,9428.490628,9422.849081,7.131986e+08,1.737635e+11,1689.0,817.0,55.0,105.0,61.0,271.0,3420.0,1671.0,11675867.0,39.0,1343.0,448.0,2237.0,124.0,330.0,331.0,2515.0,120.0,506133.0,1326610.0,1159677.0,8406185.0,281329.0,11681999.0,3.6,69.0,2.7,3.6,3.3,66.0,0.007118,1.0,606.0,2.0,1.0,1.0,2.0,5.0,4422,1.434516,2.818066e+11,9428.279323
1,ID_3239o9,1,7985.359278,7992.059917,7967.567267,4.004755e+08,1.426942e+11,920.0,544.0,20.0,531.0,103.0,533.0,1491.0,242.0,5917814.0,195.0,1070.0,671.0,3888.0,1.0,52.0,315.0,1100.0,23.0,1320.0,381117.0,1706376.0,3754815.0,80010.0,5924770.0,3.7,1.0,2.0,2.0,1.0,43.5,0.009419,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2159,-2.459507,2.126897e+11,7967.567267
2,ID_323J9k,1,49202.033778,49394.593518,49068.057046,3.017729e+09,9.166977e+11,1446.0,975.0,72.0,1152.0,187.0,905.0,9346.0,4013.0,47778746.0,104.0,2014.0,1099.0,11476.0,331.0,923.0,864.0,6786.0,442.0,9848462.0,5178557.0,2145663.0,25510267.0,5110490.0,47796942.0,3.7,22.0,3.1,3.0,3.3,65.5,0.013530,1.0,692.0,3.0,1.0,1.0,0.0,0.0,10602,4.942448,1.530712e+12,49120.738484
3,ID_323y5P,1,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.0,0.0,17.0,424.0,268.0,443.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,285,0.000000,0.000000e+00,0.000000
4,ID_324kJH,1,10535.737119,10535.737119,10384.798216,1.150053e+09,1.921183e+11,1012.0,638.0,24.0,42.0,50.0,173.0,3262.0,1652.0,14422172.0,21.0,511.0,190.0,2284.0,86.0,280.0,443.0,2284.0,169.0,311017.0,1977833.0,731277.0,10964321.0,440730.0,14426405.0,3.7,22.0,4.7,3.8,4.4,83.0,0.010332,1.0,749.0,2.0,1.0,1.0,0.0,2.0,3996,2.609576,3.386925e+11,10384.798216


In [22]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12632 entries, 0 to 12631
Data columns (total 49 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       12632 non-null  object 
 1   asset_id                 12632 non-null  int64  
 2   open                     12632 non-null  float64
 3   high                     12632 non-null  float64
 4   low                      12632 non-null  float64
 5   volume                   12632 non-null  float64
 6   market_cap               12632 non-null  float64
 7   url_shares               12632 non-null  float64
 8   unique_url_shares        12632 non-null  float64
 9   reddit_posts             12632 non-null  float64
 10  reddit_posts_score       12632 non-null  float64
 11  reddit_comments          12632 non-null  float64
 12  reddit_comments_score    12632 non-null  float64
 13  tweets                   12632 non-null  float64
 14  tweet_spam            

In [98]:
# data
col = ['open', 'high', 'low', 'market_cap', 'market_cap_global']

# features and label
features = df_train[col]
label = df_train[['close']]
print(features.shape, label.shape)

# standardization
feat_scaler = StandardScaler().fit(features)
lab_scaler = StandardScaler().fit(label)

featuresS = feat_scaler.transform(features)
labelS = lab_scaler.transform(label)

# train test split
x_train, x_test, y_train, y_test = train_test_split(featuresS, labelS, test_size=0.1, random_state=25)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(12632, 5) (12632, 1)
(11368, 5) (1264, 5) (11368, 1) (1264, 1)


In [90]:
# custom error function
def rootmse(y_true, y_pred):
    y_true = lab_scaler.inverse_transform(y_true)
    y_pred = lab_scaler.inverse_transform(y_pred)
    return mse(y_true, y_pred, squared=False)
rmse = make_scorer(rootmse, greater_is_better=False)

In [101]:
# Linear Regression
model_lr = LinearRegression()
model_lr.fit(x_train, y_train)

# score
y_pred = model_lr.predict(x_test)
print(rootmse(y_test, y_pred))

# cross validation score
score = cross_val_score(model_lr, featuresS, labelS, scoring=rmse, cv=10, n_jobs=-1)
print(min(-1*score))

58.08356785993681
45.9414115755473


In [91]:
# Random Forest 
model_rf = RandomForestRegressor()
model_rf.fit(x_train, y_train)

# score
y_pred = model_rf.predict(x_test)
print(rootmse(y_test, y_pred))

# cross validation score
score = cross_val_score(model_rf, featuresS, labelS, scoring=rmse, cv=10, n_jobs=-1)
print(min(-1*score))

50.26896596656125
43.3855775436143


In [100]:
# kfold validation
def nkfold(features=featuresS, label=labelS, fold=10, seed=42, model_param={}, verbose=1):
    """This function create random forest model with different 'fold', calculates
       its rmse score and also make prediction on test data with same model at each 
       fold """

    # additional dependencies
    from IPython.display import clear_output
    
    # model initialization
    model = RandomForestRegressor(**model_param, random_state=seed)

    # kfolds
    skf = KFold(n_splits=fold, shuffle=True, random_state=seed) 
    scores = {'rmse':[],'cv':[]}
    preds = []

    i = 0
    # looping to different fold
    for train_, test_ in skf.split(features, label):
        
        # spliting the data
        trainx, testx, trainy, testy = features[train_], features[test_], label[train_], label[test_]
        
        # prediction         
        model.fit(trainx, trainy)
        y_pred = model.predict(testx)

        # descaling labels
        testy = lab_scaler.inverse_transform(testy)
        y_pred = lab_scaler.inverse_transform(y_pred.reshape(-1, 1))

        # calculating loss
        rmse_score = mse(testy, y_pred, squared=False) # checking the accuracy

        # storing loss in list
        scores['rmse'].append(rmse_score)
        scores['cv'].append(i)

        # prediction on test data
        test_feat = feat_scaler.transform(df_test[col].fillna(0))
        test_pred = lab_scaler.inverse_transform(model.predict(test_feat).reshape(-1, 1))
        preds.append(test_pred)
        
        # print status of kfold
        i += 1
        if verbose != 0:    
            clear_output(wait=True)
            print(f'Status: {i}/{fold} ==> {(i/fold)*100}%')
        
    avg_loss = np.mean(scores['rmse'])
    return avg_loss, preds   

score, target1 = nkfold(fold=20)
score

Status: 20/20 ==> 100.0%


53.626373904995184

## Submission

In [102]:
test_features = df_test[col1]
test_features = feat_scaler.transform(test_features)

# prediction
close = model_rf.predict(test_features)
close_descale = lab_scaler.inverse_transform(close)

# sub dataframe
sub = pd.DataFrame({'id': df_test['id']})
sub['close'] = close_descale
sub.head()

,id,close
0,ID_323Sn2,6.184564e-11
1,ID_325SNW,1.134900e+04
2,ID_325uzE,6.314281e+03
3,ID_328qCx,6.184564e-11
4,ID_3293uJ,6.184564e-11


In [103]:
sub.to_csv('ml_sub1.csv', index=False)